In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [3]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [4]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=feature_extractor.sampling_rate))

In [6]:
gtzan["train"][0]

{'file': '/home/lyc/.cache/huggingface/datasets/downloads/extracted/3a38d5ea7fbd527511ae15290740652c26d8d707b9901c5c30c6cd145facb53f/genres/pop/pop.00098.wav',
 'audio': {'path': '/home/lyc/.cache/huggingface/datasets/downloads/extracted/3a38d5ea7fbd527511ae15290740652c26d8d707b9901c5c30c6cd145facb53f/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427], shape=(480003,)),
  'sampling_rate': 16000},
 'genre': 7}

In [7]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [8]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
)
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [9]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [10]:
from transformers import AutoModelForAudioClassification
from itertools import count

# id2label_fn = gtzan["train"].features["genre"].int2str

# id2label = {
#     i: id2label_fn(i)
#     for i in range(len(gtzan_encoded["train"].features["label"].names))
# }
id2label = {
    i: v for i, v in zip(count(), gtzan_encoded["train"].features["label"].names)
}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 32
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    metric_for_best_model="accuracy",
    fp16=True,
)

In [12]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_1074148/2059408770.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
5,1.762900
10,1.770700
15,1.749500
20,1.717000
25,1.620700
30,1.632800
35,1.501800
40,1.497200
45,1.398900
50,1.394100


TrainOutput(global_step=290, training_loss=0.9055602188768058, metrics={'train_runtime': 2376.4566, 'train_samples_per_second': 3.783, 'train_steps_per_second': 0.122, 'total_flos': 6.133988274624e+17, 'train_loss': 0.9055602188768058, 'epoch': 10.0})